In [1]:
from importlib import reload

import datasets
import numpy as np

import src.models.components.feature_extractor_dinov2
import src.models.components.spatiotemporal_encoder

In [2]:
rwth_phoenix = datasets.load_dataset('lukasbraach/rwth_phoenix_weather_2014', 'multisigner', streaming=True)
it = iter(rwth_phoenix['train'])
first = next(it)

In [13]:
reload(src.models.components.feature_extractor_dinov2)
from src.models.components.feature_extractor_dinov2 import SignLanguageFeatureExtractor

feature_extractor = SignLanguageFeatureExtractor()

src_tensor = [
    np.random.rand(3, 3, 224, 224),
    np.random.rand(1, 3, 224, 224),
    np.random.rand(6, 3, 224, 224),
]
batched_feature = feature_extractor(src_tensor, sampling_rate=25, pad_to_multiple_of=32, return_tensors="pt")

input_values = batched_feature.input_values
attention_mask = batched_feature.attention_mask

print("input_values:", input_values.shape)
print("attention_mask:", attention_mask)

input_values: torch.Size([3, 32, 768])
attention_mask: tensor([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]], dtype=torch.int32)


In [42]:
reload(src.models.components.spatiotemporal_encoder)
from src.models.components.spatiotemporal_encoder import SpatiotemporalEncoder
from src.models.components.spatiotemporal_encoder import SpatiotemporalEncoderConfig

config = SpatiotemporalEncoderConfig()

model = SpatiotemporalEncoder(config)
encoded = model(input_values, attention_mask)

encoded

Wav2Vec2BaseModelOutput(last_hidden_state=tensor([[[-1.3873,  1.4464,  0.6032,  ...,  0.5986,  0.6305, -1.9885],
         [-0.7752,  1.3980,  1.1602,  ..., -0.3088, -0.1042, -1.0873],
         [-0.9764,  1.5781,  1.1313,  ...,  0.4113, -0.3345, -1.8981],
         ...,
         [-0.7431,  1.3568, -0.0080,  ...,  0.9427,  0.2413, -0.6780],
         [-0.7913,  1.1805, -0.2928,  ...,  1.3326,  0.5124,  0.2337],
         [-0.4308,  1.3446,  0.4378,  ...,  1.2666,  0.1246, -0.1829]],

        [[-1.4415, -1.2161,  0.4831,  ...,  0.9675,  0.6535,  1.0248],
         [ 0.5653,  0.7371,  2.0587,  ..., -0.4363,  1.0376,  1.0097],
         [ 0.5841,  0.9852, -0.4640,  ...,  0.0693, -0.6144,  0.6243],
         ...,
         [-0.9704, -1.0217,  0.0294,  ...,  0.8167,  0.6037, -0.2885],
         [ 0.7069,  1.1435, -1.9218,  ...,  0.5191, -1.1076, -1.9945],
         [ 1.0791,  1.1473, -0.2925,  ..., -0.2680, -0.1915, -1.0578]],

        [[ 0.1232,  0.8341,  1.4454,  ..., -0.0923,  0.4459,  1.5432],
   

In [45]:
reload(src.models.components.sign_language_net)
from src.models.components.sign_language_net import SignLanguageNet


model = SignLanguageNet()

model

SignLanguageNet(
  (encoder): SpatiotemporalEncoder(
    (feature_extractor): SpatialFeatureEncoder()
    (feature_projection): SpatiotemporalFeatureProjection(
      (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): Wav2Vec2Encoder(
      (pos_conv_embed): Wav2Vec2PositionalConvEmbedding(
        (conv): Conv1d(768, 768, kernel_size=(128,), stride=(1,), padding=(64,), groups=16)
        (padding): Wav2Vec2SamePadLayer()
        (activation): GELUActivation()
      )
      (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (layers): ModuleList(
        (0-5): 6 x Wav2Vec2EncoderLayer(
          (attention): Wav2Vec2Attention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=7

In [44]:
out = model.generate(inputs=input_values, attention_mask=attention_mask, max_length=100)

out

tensor([[2, 2],
        [2, 2],
        [2, 2]])